# Метрика TF-IDF

**TF-IDF (Term Frequency-Inverse Document Frequency)** — это метрика, используемая в текстовой аналитике для оценки важности слова в документе относительно всего корпуса документов.

- **TF (Term Frequency)**: Частота слова в документе. Чем чаще слово встречается в документе, тем выше его значение TF. Если слово "машина" встречается в документе 5 раз, а в документе всего 100 слов, то TF("машина") = 5/100 = 0.05.
- **IDF (Inverse Document Frequency)**: Обратная частота документа. Эта часть метрики указывает на редкость слова в корпусе документов. Чем реже слово встречается в других документах, тем выше его значение IDF. Если слово "машина" встречается в 3 из 100 документов, то IDF("машина") = log(100/3).

TF-IDF рассчитывается как произведение TF и IDF:

TF-IDF(t,d)=TF(t,d)×IDF(t), где t — слово, d — документ, а N — общее число документов.

Предположим, у нас есть 4 документа и мы хотим оценить важность слова "машина":

TF:
- В документе 1 слово "машина" встречается 5 раз из 100 слов, TF = 0.05.
- В документе 2 слово "машина" встречается 3 раза из 50 слов, TF = 0.06.
  
IDF:
- Если "машина" встречается в 3 из 4 документов, IDF = log(4/3).
  
TF-IDF:
- Для документа 1: TF-IDF = 0.05 * log(4/3).
- Для документа 2: TF-IDF = 0.06 * log(4/3).
  
TF-IDF используется в задачах:
- Поиск информации: TF-IDF помогает найти наиболее релевантные документы по запросу пользователя. Чем выше TF-IDF слова в документе, тем более важным считается это слово для данного документа.
- Тематическое моделирование: Определение тем в тексте и кластеризация документов по темам.
- Анализ текста: Помогает в выявлении ключевых слов и фраз в больших объемах текста.

## Рассчёт TF-IDF корпуса документов

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd

import pymorphy2
import string

In [ ]:
documents = [
    "В последние десятилетия интеллектуальные технологии стали неотъемлемой частью многих сфер жизни, включая образование, здравоохранение, финансы и промышленность. Машинное обучение и искусственный интеллект продолжают развиваться, внедряясь в различные аспекты нашего общества и экономики. Рост объема данных, доступных для анализа, создает большие возможности для применения технологий машинного обучения и искусственного интеллекта в повседневной жизни.",
    "Современные системы искусственного интеллекта, такие как нейронные сети и глубокое обучение, обладают потенциалом для автоматизации процессов принятия решений, улучшения производительности и предоставления персонализированных услуг. Однако с ростом использования этих технологий возникают новые вызовы, такие как проблемы конфиденциальности данных, этические вопросы и неопределенность в прогнозах.",
    "Подходы к решению этих проблем включают в себя разработку новых методов обработки и анализа данных, улучшение алгоритмов машинного обучения и создание строгих правил для защиты конфиденциальности и безопасности данных. Кроме того, важно проводить обучение и образование в области искусственного интеллекта, чтобы повысить осведомленность и компетентность в этой области.",
    "Одним из ключевых направлений развития искусственного интеллекта является создание автономных систем, способных принимать решения и действовать без участия человека. Эти системы могут использоваться в различных областях, таких как автопилоты для автомобилей, управление трафиком, медицинские диагнозы и рекомендательные системы.",
    "Однако внедрение автономных систем также вызывает вопросы безопасности и этики, связанные с возможностью ошибок и неспособностью системы объяснить свои решения. Поэтому важно разрабатывать методы верификации и валидации автономных систем, а также устанавливать четкие правила использования и ответственности за их действия."
]

In [ ]:
documents = [
    "Машина едет по дороге.",
    "Водитель управляет машиной.",
    "Дорога прямая и свободная.",
    "Машина остановилась на светофоре."
]

In [ ]:
# Загрузка стоп-слов и установка токенизатора
stop_words = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()

# Функция для лемматизации текста
def lemmatize(text):
    tokens = word_tokenize(text.lower())
    lemmas = [morph.parse(token)[0].normal_form for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(lemmas)

processed_documents = [lemmatize(doc) for doc in documents]

# Применение TF-IDF для трансформации корпуса
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_documents)

# Получение списка слов
feature_names = vectorizer.get_feature_names_out()

# Преобразование TF-IDF матрицы в DataFrame для удобства отображения
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Вывод результатов
display(tfidf_df.style.format(precision=4))

### Определение ключевых слов для каждого документа 

In [ ]:
# Функция для определения ключевых слов для каждого документа
def get_top_keywords(tfidf_matrix, feature_names, top_n=5):
    top_keywords = []
    for row in tfidf_matrix:
        row_array = row.toarray().flatten()
        top_indices = row_array.argsort()[-top_n:][::-1]
        top_words = [feature_names[idx] for idx in top_indices]
        top_keywords.append(top_words)
    return top_keywords
    
# Определение ключевых слов
top_keywords = get_top_keywords(tfidf_matrix, feature_names)

# Вывод ключевых слов для каждого документа
for doc_index, keywords in enumerate(top_keywords):
    print(f"{doc_index + 1}. Ключевые слова: {', '.join(keywords)}\n")

### Определение сходства документов

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_similarities, index=[f"Документ {i+1}" for i in range(len(documents))], columns=[f"Документ {i+1}" for i in range(len(documents))])
print("Сходство документов:\n")
print(cosine_sim_df)

### Создание облака слов для каждого документа

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Функция для создания облака слов для каждого документа
def create_wordcloud(text, title):
    wordcloud = WordCloud(width=400, height=300, background_color='white').generate(text)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)

# Визуализация облаков слов в виде сетки
num_docs = len(documents)
grid_size = int(np.ceil(np.sqrt(num_docs)))
fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 10))

for i, doc in enumerate(processed_documents):
    row = i // grid_size
    col = i % grid_size
    ax = axes[row, col]
    wordcloud = WordCloud(width=400, height=300, background_color='white').generate(doc)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')
    ax.set_title(f"Документ {i+1}")

# Удаление пустых осей
for i in range(num_docs, grid_size * grid_size):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()

### Создание общего текста для облака слов

In [ ]:

combined_text = ' '.join(processed_documents)

# Визуализация общего облака слов
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(combined_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Общее облако слов для всех документов")
plt.show()

## Темы исследовательских работ

1. Анализ ключевых слов в текстах разных жанров

Ученики могут собрать тексты различных жанров (например, новости, научные статьи, художественная литература) и использовать TF-IDF для определения ключевых слов в каждом жанре.

Исследовательские вопросы:
- Какие слова наиболее часто встречаются в новостных статьях по сравнению с научными статьями?
- Как различаются ключевые слова в художественной литературе и научной литературе?

2. Изменение ключевых слов с течением времени

Ученики могут взять тексты, написанные в разные исторические периоды (например, газеты или книги) и проанализировать, как менялись ключевые слова с течением времени.

Исследовательские вопросы:
- Какие слова были популярны в разное время?
- Как изменялись ключевые темы и понятия с течением времени?

3. Сравнение ключевых слов в текстах разных авторов

Ученики могут сравнить тексты разных авторов, чтобы понять уникальные стили и темы, которые они используют.

Исследовательские вопросы:
- Какие слова и темы уникальны для каждого автора?
- Как различаются ключевые слова между авторами одного жанра?

4. Анализ текста социальных сетей

Ученики могут анализировать тексты из социальных сетей (например, посты в Twitter или Instagram) для выявления популярных тем и слов.

Исследовательские вопросы:
- Какие темы наиболее популярны в социальных сетях?
- Как меняются популярные темы со временем?

5. Выявление плагиата

Ученики могут использовать TF-IDF для выявления плагиата, сравнивая тексты и определяя, насколько они похожи.

Исследовательские вопросы:
- Насколько схожи два текста по своим ключевым словам?
- Можно ли определить плагиат, используя TF-IDF?